In [368]:
import pandas as pd
import numpy as np
import math

from tqdm import tqdm
import time

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

In [369]:
RANDOM_SEED = 13022022

In [370]:
new_df = pd.read_csv('new_df.csv')

In [371]:
new_df.sample()

,Active,train,len,4,5,6,7,8,9,10,...,701,702,703,704,705,706,707,708,709,710
5588,0,0,40,45,44,43,42,43,43,43,...,0,0,0,0,0,0,0,0,0,0


In [372]:
train_new = new_df[new_df['train'] == 1]
test_new =new_df[new_df['train'] == 0]

In [373]:
X = train_new.drop(['Active', 'train'], axis = 1)
y = train_new['Active']

In [374]:
X_test = test_new.drop(['Active', 'train'], axis = 1)

In [375]:
X.sample()

,len,4,5,6,7,8,9,10,11,12,...,701,702,703,704,705,706,707,708,709,710
419,58,40,35,25,37,30,40,35,19,37,...,0,0,0,0,0,0,0,0,0,0


In [376]:
features = np.array(X)

In [377]:
test_features = np.array(X_test)

In [378]:
x_test = torch.tensor(test_features)
x_test = x_test.type(torch.FloatTensor)
x_test.size()

torch.Size([1614, 708])

In [379]:
x_train = torch.tensor(features)
x_train = x_train.type(torch.FloatTensor)
x_train.size()

torch.Size([5557, 708])

In [380]:
x_labels = torch.tensor(y)
x_labels = x_labels.type(torch.FloatTensor)
x_labels.size()

torch.Size([5557])

In [381]:
train_labels

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [382]:
nX = 708
nH1 = 500
nH2 = 350
nH3 = 150
nH4 = 75
nH5 = 30
nH6 = 15
nH7 = 7
nY = 1
 
class TwoLayersNet(nn.Module):
    def __init__(self, nX, nH1, nH2, nH3, nH4, nH5, nH6, nH7, nY):        
        super(TwoLayersNet, self).__init__()     # конструктор предка с этим именем
         
        self.fc1 = nn.Linear(nX, nH1)             # создаём параметры модели
        self.fc2 = nn.Linear(nH1, nH2)             # создаём параметры модели
        self.fc3 = nn.Linear(nH2, nH3)             # создаём параметры модели
        self.fc4 = nn.Linear(nH3, nH4)             # создаём параметры модели
        self.fc5 = nn.Linear(nH4, nH5)             # создаём параметры модели
        self.fc6 = nn.Linear(nH5, nH6)             # создаём параметры модели
        self.fc7 = nn.Linear(nH6, nH7)             # создаём параметры модели
        self.fc8 = nn.Linear(nH7, nY)             # в полносвязных слоях
          
    def forward(self, x):                        # задаётся прямой проход
        x = self.fc1(x)                          # выход первого слоя
        x = nn.Sigmoid()(x)                      # пропускаем через Sigmoid
        x = self.fc2(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc3(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc4(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc5(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc6(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc7(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc8(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        return x
          
model = TwoLayersNet(nX, nH1, nH2, nH3, nH4, nH5, nH6, nH7, nY)

In [383]:
model = TwoLayersNet(nX, nH1, nH2, nH3, nH4, nH5, nH6, nH7, nY)                           # экземпляр сети        
 
loss      = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),          # параметры модели
                            lr=0.5, momentum=0.8) 

In [384]:
def fit(model, x_train,x_labels, batch_size=100, train=True):    
    model.train(train)                                 # важно для Dropout, BatchNorm
    sumL, sumA, numB = 0, 0, int( len(x_train)/batch_size )  # ошибка, точность, батчей
       
    for i in range(0, numB*batch_size, batch_size):          
        xb = x_train[i: i+batch_size]                          # текущий батч,
        yb = x_labels[i: i+batch_size]                          # X,Y - torch тензоры
              
        y = model(xb)[:,0]                                    # прямое распространение

        L = loss(y, yb)                                  # вычисляем ошибку
  
        if train:                                        # в режиме обучения
            optimizer.zero_grad()                        # обнуляем градиенты        
            L.backward()                                 # вычисляем градиенты            
            optimizer.step()                             # подправляем параметры
                                     
        sumL += L.item()                                 # суммарная ошибка (item из графа)
        sumA += (y.round() == yb).float().mean()         # точность определения класса
         
    return sumL/numB,  sumA/numB 

In [385]:
print( "before:      loss: %.4f accuracy: %.4f" %  fit(model, x_train,x_labels, train=False) )
 
epochs = 1                                            # число эпох
for epoch in tqdm(range(epochs)):                              # эпоха - проход по всем примерам
    L,A = fit(model, x_train, x_labels)                               # одна эпоха
     
    if epoch % 500 == 0 or epoch == epochs-1:                 
        print(f'epoch: {epoch:5d} loss: {L:.4f} accuracy: {A:.4f}' ) 

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

before:      loss: 0.6596 accuracy: 0.9629


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.04s/it]

epoch:     0 loss: 0.1790 accuracy: 0.9629


In [386]:
y_pred = model(x_test)

In [387]:
y_pred = y_pred[:,0]

In [388]:
y_pred

tensor([0.0446, 0.0446, 0.0446,  ..., 0.0446, 0.0446, 0.0446],
       grad_fn=<SelectBackward0>)

In [349]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [389]:
test['Active'] = y_pred.detach().numpy()

In [390]:
test.sample()

,Smiles,Active
1167,CC(C)(O)CC(=O)O.CC(C)(O)CC(=O)O,0.044628


In [392]:
test['Active'] = test['Active'].apply(lambda x: x*1000000-44628)

In [394]:
test['Active'] = test['Active'].apply(lambda x: int(abs(round(x, 0))))

In [395]:
test['Active'].value_counts()

0    1513
1     101
Name: Active, dtype: int64

In [396]:
test.sample(5)

,Smiles,Active
602,N=C(NCCCCCCNC(=N)NC(=N)Nc1ccc(Cl)cc1)NC(=N)Nc1...,0
478,Cc1c(C)c2c(c(C)c1OC(=O)CCC(=O)O)CC[C@@](C)(CCC...,0
64,NC(=O)c1cnc(N[C@@H]2CCCC[C@@H]2N)nc1Nc1cccc(-n...,0
1160,CC(C)(C)c1cc(/C=C(\C#N)C(N)=S)cc(C(C)(C)C)c1O,0
205,C[N+](C)(C)CCOP(=O)(O)OP(=O)(O)OC[C@H]1O[C@@H]...,1


In [397]:
test.to_csv('test.csv', index=False)